In [ ]:
import pandas as pd

from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

In [2]:
df = pd.read_csv('data/prepared_data_descriptions.csv')
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,Status,Description_Modified_00,Description_Modified_01,StockCode_Modified,Description_Modified_02
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom,Completed,WHITE HANGING HEART T-LIGHT HOLDER,WHITE HANGING HEART T-LIGHT HOLDER,85123A,WHITE HANGING HEART T-LIGHT HOLDER
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,Completed,WHITE METAL LANTERN,WHITE METAL LANTERN,71053,WHITE METAL LANTERN
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom,Completed,CREAM CUPID HEARTS COAT HANGER,CREAM CUPID HEARTS COAT HANGER,84406B,CREAM CUPID HEARTS COAT HANGER
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,Completed,KNITTED UNION FLAG HOT WATER BOTTLE,KNITTED UNION FLAG HOT WATER BOTTLE,84029G,KNITTED UNION FLAG HOT WATER BOTTLE
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,Completed,RED WOOLLY HOTTIE WHITE HEART,RED WOOLLY HOTTIE WHITE HEART,84029E,RED WOOLLY HOTTIE WHITE HEART


In [3]:
df = df[df["Status"] == "Completed"]
df = df[df['Quantity'] > 0]
df.dropna(subset=['InvoiceNo', 'Description_Modified_01'], inplace=True)
df.reset_index(drop=True, inplace=True)

1. Create the Transaction-Item Matrix
- The Apriori algorithm requires data in a "one-hot encoded" format. This means we need a matrix where:
    - Each row is a single transaction (InvoiceNo).
    - Each column is a unique product (StockCode).
- The value is 1 if the product was in the basket and 0 if it was not.

In [4]:
# Group by InvoiceNo and Description_Modified_01, summing quantities (though we only care if it's > 0)
basket = df.groupby(['InvoiceNo', 'Description_Modified_01'])['Quantity'].sum().unstack().reset_index().fillna(0).set_index('InvoiceNo')

# Binarize the data: 1 if item was purchased (Quantity > 0), 0 otherwise
def encode_units(x):
    if x <= 0:
        return 0
    if x >= 1:
        return 1

basket_sets = basket.applymap(encode_units)
basket_sets.head()

C:\Users\PREDATOR HELIOS 300\AppData\Local\Temp\ipykernel_9232\2563449534.py:11: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  basket_sets = basket.applymap(encode_units)


Description_Modified_01  10 COLOUR SPACEBOY PEN  12 COLOURED PARTY BALLOONS  \
InvoiceNo                                                                     
536365                                        0                           0   
536366                                        0                           0   
536367                                        0                           0   
536368                                        0                           0   
536369                                        0                           0   

Description_Modified_01  12 DAISY PEGS IN WOOD BOX  12 EGG HOUSE PAINTED WOOD  \
InvoiceNo                                                                       
536365                                           0                          0   
536366                                           0                          0   
536367                                           0                          0   
536368                                           0                          0   
536369                                           0                          0   

Description_Modified_01  12 HANGING EGGS HAND PAINTED  \
InvoiceNo                                               
536365                                              0   
536366                                              0   
536367                                              0   
536368                                              0   
536369                                              0   

Description_Modified_01  12 IVORY ROSE PEG PLACE SETTINGS  \
InvoiceNo                                                   
536365                                                  0   
536366                                                  0   
536367                                                  0   
536368                                                  0   
536369                                                  0   

Description_Modified_01  12 MESSAGE CARDS WITH ENVELOPES  \
InvoiceNo                                                  
536365                                                 0   
536366                                                 0   
536367                                                 0   
536368                                                 0   
536369                                                 0   

Description_Modified_01  12 PENCIL SMALL TUBE WOODLAND  \
InvoiceNo                                                
536365                                               0   
536366                                               0   
536367                                               0   
536368                                               0   
536369                                               0   

Description_Modified_01  12 PENCILS SMALL TUBE RED RETROSPOT  \
InvoiceNo                                                      
536365                                                     0   
536366                                                     0   
536367                                                     0   
536368                                                     0   
536369                                                     0   

Description_Modified_01  12 PENCILS SMALL TUBE SKULL  \
InvoiceNo                                              
536365                                             0   
536366                                             0   
536367                                             0   
536368                                             0   
536369                                             0   

Description_Modified_01  12 PENCILS TALL TUBE POSY  \
InvoiceNo                                            
536365                                           0   
536366                                           0   
536367                                           0   
536368                                           0   
536369                                           0   

D

2. Generate Frequent Itemsets (Apriori)
- Use the apriori function to find groups of items that are frequently purchased together.
    - min_support: Minimum percentage of all transactions that must contain an itemset for it to be considered "frequent."

In [ ]:
# Generate frequent itemsets
frequent_itemsets = apriori(basket_sets, min_support=0.01, use_colnames=True, low_memory=True)
frequent_itemsets.sort_values(by='support', ascending=False).head()

c:\Users\PREDATOR HELIOS 300\miniconda3\envs\rei-ecommerce\Lib\site-packages\mlxtend\frequent_patterns\fpcommon.py:161: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(


,support,itemsets
594,0.107611,(WHITE HANGING HEART T-LIGHT HOLDER)
416,0.092541,(REGENCY CAKESTAND 3 TIER)
250,0.086992,(JUMBO BAG RED RETROSPOT)
340,0.075023,(PARTY BUNTING)
36,0.074806,(ASSORTED COLOUR BIRD ORNAMENT)


3. Generate Association Rules
- From the list of frequent itemsets, generate the actual association rules (e.g., "IF item A, THEN item B").
    - metric: The metric used to filter rules.
    - min_threshold: The minimum value for that metric. For lift, a value of 1 is standard, as it means the items have at least some positive association.

In [ ]:
# Generate the rules
rules = association_rules(frequent_itemsets, metric="lift", min_threshold=1)

rules.sort_values(by=['lift', 'confidence'], ascending=False, inplace=True)
rules.head(10)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,representativity,leverage,conviction,zhangs_metric,jaccard,certainty,kulczynski
1149,(REGENCY TEA PLATE PINK),"(REGENCY TEA PLATE ROSES, REGENCY TEA PLATE GREEN)",0.012186,0.012459,0.010065,0.825893,66.291426,1.0,0.009913,5.672033,0.997066,0.690299,0.823696,0.816877
1144,"(REGENCY TEA PLATE ROSES, REGENCY TEA PLATE GREEN)",(REGENCY TEA PLATE PINK),0.012459,0.012186,0.010065,0.807860,66.291426,1.0,0.009913,5.141120,0.997340,0.690299,0.805490,0.816877
1148,(REGENCY TEA PLATE GREEN),"(REGENCY TEA PLATE ROSES, REGENCY TEA PLATE PINK)",0.014689,0.010718,0.010065,0.685185,63.930908,1.0,0.009907,3.142426,0.999033,0.656028,0.681775,0.812136
1145,"(REGENCY TEA PLATE ROSES, REGENCY TEA PLATE PINK)",(REGENCY TEA PLATE GREEN),0.010718,0.014689,0.010065,0.939086,63.930908,1.0,0.009907,16.175521,0.995022,0.656028,0.938178,0.812136
607,(REGENCY TEA PLATE PINK),(REGENCY TEA PLATE GREEN),0.012186,0.014689,0.010990,0.901786,61.391567,1.0,0.010811,10.032257,0.995847,0.691781,0.900322,0.824967
606,(REGENCY TEA PLATE GREEN),(REGENCY TEA PLATE PINK),0.014689,0.012186,0.010990,0.748148,61.391567,1.0,0.010811,3.922201,0.998376,0.691781,0.745041,0.824967
1140,"(POPPY'S PLAYHOUSE KITCHEN, POPPY'S PLAYHOUSE BEDROOM)",(POPPY'S PLAYHOUSE LIVINGROOM),0.013819,0.013710,0.010119,0.732283,53.413105,1.0,0.009930,3.684084,0.995028,0.581250,0.728562,0.735189
1141,(POPPY'S PLAYHOUSE LIVINGROOM),"(POPPY'S PLAYHOUSE KITCHEN, POPPY'S PLAYHOUSE BEDROOM)",0.013710,0.013819,0.010119,0.738095,53.413105,1.0,0.009930,3.765420,0.994918,0.581250,0.734425,0.735189
600,(REGENCY SUGAR BOWL GREEN),(REGENCY MILK JUG PINK),0.014580,0.014798,0.011207,0.768657,51.943673,1.0,0.010991,4.258616,0.995260,0.616766,0.765182,0.763005
601,(REGENCY MILK JUG PINK),(REGENCY SUGAR BOWL GREEN),0.014798,0.014580,0.011207,0.757353,51.943673,1.0,0.010991,4.061124,0.995479,0.616766,0.753763,0.763005


### Definitions:

- antecedents (IF): The item(s) a customer puts in their basket (e.g., {'LUNCH BAG RED RETROSPOT'})
- consequents (THEN): The item(s) the customer is also likely to buy (e.g., {'LUNCH BAG PINK POLKADOT'})
- support (Popularity): Percentage of all transactions that contain both items.
- confidence (Reliability): Of all customers who bought the "IF" item, what percentage also bought the "THEN" item.
- lift (Strength): How much more likely a customer is to buy the "THEN" item, given they bought the "IF" item.

### Interpretations:
- Lift > 1: Positive association. 
- Lift = 1: No association.
- Lift < 1: Negative association.

### Business Actions:
Look for rules with high Lift and high Confidence. These are the most actionable insights for product bundling.